# import, data load

In [1]:
COLUMNS = ["user_id" ,"trip", "course", "place_id","place_name", "place_category","rating", "review", "place_address", "place_lat", "place_lng","y", "belief", "y_pred"]

In [2]:
!pip install haversine

In [3]:
import pandas as pd
import numpy as np
from haversine import haversine
import itertools

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data_couple_before = pd.read_csv('/content/drive/MyDrive/deepFM_result_data2/couple_b.csv', names = COLUMNS, header=None, skiprows = 1)
data_couple_after = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/couple_b.csv', names = COLUMNS, header=None, skiprows = 1)

data_family_before = pd.read_csv('/content/drive/MyDrive/deepFM_result_data2/family_b.csv', names = COLUMNS, header=None, skiprows = 1)
data_family_after = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/family_b.csv', names = COLUMNS, header=None, skiprows = 1)

data_hotel_before = pd.read_csv('/content/drive/MyDrive/deepFM_result_data2/hotel_b.csv', names = COLUMNS, header=None, skiprows = 1)
data_hotel_after = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/hotel_b.csv', names = COLUMNS, header=None, skiprows = 1)

data_guesthouse_before = pd.read_csv('/content/drive/MyDrive/deepFM_result_data2/guesthouse_b.csv', names = COLUMNS, header=None, skiprows = 1)
data_guesthouse_after = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/guesthouse_b.csv', names = COLUMNS, header=None, skiprows = 1)

data_pension_before = pd.read_csv('/content/drive/MyDrive/deepFM_result_data2/pension_b.csv', names = COLUMNS, header=None, skiprows = 1)
data_pension_after = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/pension_b.csv', names = COLUMNS, header=None, skiprows = 1)

# 함수

In [8]:
def min_distance(user_number, data):
  loc_1 = (data.loc[user_number]['place_lat'], data.loc[user_number]['place_lng'])
  loc_2 = (data.loc[user_number+1]['place_lat'], data.loc[user_number+1]['place_lng'])
  loc_3 = (data.loc[user_number+2]['place_lat'], data.loc[user_number+2]['place_lng'])
  loc_4 = (data.loc[user_number+3]['place_lat'], data.loc[user_number+3]['place_lng'])
  loc_5 = (data.loc[user_number+4]['place_lat'], data.loc[user_number+4]['place_lng'])
  result = list(itertools.permutations([loc_1, loc_2, loc_3, loc_4, loc_5], 5))
  result_str = list(itertools.permutations(["1", "2", "3", "4", "5"], 5))
  min_dis = 0
  course = []
  n = len(result)
  for i in range(0, n):
    dis = haversine(result[i][0],result[i][1]) + haversine(result[i][1],result[i][2]) + haversine(result[i][2],result[i][3]) + haversine(result[i][3],result[i][4])
    if(i==0):
      min_dis = dis
    if(dis < min_dis):
      min_dis = dis
      course = result_str[i]
  return min_dis , course

In [9]:
def get_distance_table(data_before, data_after):
  distance_dataframe = pd.DataFrame(columns=['user_id',	'before_addr', 'before_course',	'after_addr', "after_course", "diff", "ratio", "diff_accumulate"])
  diff_accumu = 0
  for i in range(0, 50):
    user_index = i * 5 
    before_dis, before_course = min_distance(user_index, data_before)
    before_course_str = " ".join(before_course)
    after_dis, after_course = min_distance(user_index, data_after)
    after_course_str = " ".join(after_course)
    distance_dataframe = distance_dataframe.append({'user_id' : str(data_before.loc[user_index]['user_id'])}, ignore_index=True)
    distance_dataframe.loc[i]['before_addr'] = before_dis
    distance_dataframe.loc[i]['before_course'] = before_course_str
    distance_dataframe.loc[i]['after_addr'] = after_dis
    distance_dataframe.loc[i]['after_course'] = after_course_str
    diff = before_dis - after_dis
    distance_dataframe.loc[i]['diff'] = diff
    ratio = after_dis/before_dis
    distance_dataframe.loc[i]['ratio'] = ratio
    diff_accumu = diff_accumu + diff
    distance_dataframe.loc[i]['diff_accumulate'] = diff_accumu 

  return distance_dataframe

# 카테고리별 거리 표 구하기

In [13]:
couple_distance_table = pd.DataFrame(columns=['user_id',	'before_addr','before_course',	'after_addr', "after_course", "diff", "ratio", "diff_accumulate"])
couple_distance_table = get_distance_table(data_couple_before, data_couple_after)
couple_distance_table.to_csv('/content/drive/MyDrive/deepFM_RESULT/deepFM_couple.xlsx',index=False)

In [14]:
family_distance_table = pd.DataFrame(columns=['user_id',	'before_addr','before_course',	'after_addr', "after_course", "diff", "ratio", "diff_accumulate"])
family_distance_table = get_distance_table(data_family_before, data_family_after)
family_distance_table.to_csv('/content/drive/MyDrive/deepFM_RESULT/deepFM_family.xlsx',index=False)

In [15]:
hotel_distance_table = pd.DataFrame(columns=['user_id',	'before_addr','before_course',	'after_addr', "after_course", "diff", "ratio", "diff_accumulate"])
hotel_distance_table = get_distance_table(data_hotel_before, data_hotel_after)
hotel_distance_table.to_csv('/content/drive/MyDrive/deepFM_RESULT/deepFM_hotel.xlsx',index=False)

In [16]:
guesthouse_distance_table = pd.DataFrame(columns=['user_id',	'before_addr','before_course',	'after_addr', "after_course", "diff", "ratio", "diff_accumulate"])
guesthouse_distance_table = get_distance_table(data_guesthouse_before, data_guesthouse_after)
guesthouse_distance_table.to_csv('/content/drive/MyDrive/deepFM_RESULT/deepFM_guesthouse.xlsx',index=False)

In [17]:
pension_distance_table = pd.DataFrame(columns=['user_id',	'before_addr','before_course',	'after_addr', "after_course", "diff", "ratio", "diff_accumulate"])
pension_distance_table = get_distance_table(data_pension_before, data_pension_after)
pension_distance_table.to_csv('/content/drive/MyDrive/deepFM_RESULT/deepFM_pension.xlsx',index=False)